In [ ]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import cv2, os, math, time, sys
from datetime import timedelta
from sklearn.utils import shuffle
sys.path.append('../../original_images')
from gen_data_batch import generate_batch

In [ ]:
###Configuration
"""
Data Configurations/Paths
"""
img_dir_patch="./SD/predicted_patches"
img_dir_orig = "../../original_images/SD"

modelsd_test = 'SD/modelsd_test.ckpt'
# img_type = "original"
img_type = "patch"

##
# Convolutional Layer 1.
filter_size1 = 4          # Convolution filters are 4 x 4 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 2          # Convolution filters are 2 x 2 pixels.
num_filters2 = 32         # There are 32 of these filters.

# Convolutional Layer 3.
filter_size3 = 2          # Convolution filters are 2 x 2 pixels.
num_filters3 = 64         # There are 64 of these filters.

# Convolutional Layer 4.
filter_size4 = 2          # Convolution filters are 2 x 2 pixels.
num_filters4 = 128         # There are 128 of these filters.

# Fully-connected layer.
fc_size = 256             # Number of neurons in fully-connected layer.

# Number of colour channels for the images: 3 channel for RGB.
num_channels = 3

# Tuple with height and width of images used to reshape arrays.
img_shape = (60, 60, num_channels)
item_size = (5,5)
# Number of classes, one class for same and one for different image
num_classes = 2
nitems = 2

In [ ]:
train_batch_size = 64
train_data, mask_labels, left_mask, right_mask, labels = generate_batch(train_batch_size, img_shape, item_size, nitems)
print(train_data.shape)
print(mask_labels.shape)
print(left_mask.shape)
print(right_mask.shape)
see_output(np.reshape(train_data, [64, 60,60,3])[:1, :,:,:])
see_output_grey(mask_labels[:1, :,:])
see_output_grey(left_mask[:1, :, :])
see_output_grey(right_mask[:1, :, :])

In [ ]:
def load_data(img_dir, img_type="patch"):
        list_of_imgs = []
        list_same_diff = []
        for img_no in os.listdir(img_dir):
            img_no_path = os.path.join(img_dir, img_no)
            for img_label in os.listdir(img_no_path):
                    
                list_same_diff.append(int(img_label))
                img_lbl_path = os.path.join(img_no_path, img_label)
#                 print(img_lbl_path)
                if img_type == "original":
                    img_lbl_path = img_lbl_path + "/img/"
                    
                if img_type == "patch":
                    for img in os.listdir(img_lbl_path):
#                         if img == "labels":
                        img_path = os.path.join(img_lbl_path, img)
#                             img_path = img_path + "/merged_patch.png"
                        list_of_imgs.append(img_path)
                else:    
                    for img in os.listdir(img_lbl_path):
                        img_path = os.path.join(img_lbl_path, img)
                        list_of_imgs.append(img_path)
#         print(list_of_imgs)
        data_imgs = np.array(list_of_imgs)
        data_same_diff = np.array(list_same_diff)

        return data_imgs, data_same_diff
    
def get_batch_images(data, same_diff, type_img = "patch"):
        list_of_imgs = []
        list_of_same_diff = []
        for img, img_type in zip(data, same_diff):
            orig_img = cv2.imread(img)
            #only first image as a label
            if orig_img is None:
                    print ("Unable to read image{}".format(img))
                    continue
            
            if type_img == "original":
                flattened_img = orig_img.flatten()
                list_of_imgs.append(np.asarray(flattened_img, dtype=np.float32))
                
                if img_type == 1: #0 is same and 1 is different
                    list_of_same_diff.append([0,1])
                else:
                    list_of_same_diff.append([1,0])
            else:            
                if orig_img.shape == (4, 2, 3):
                    flattened_img = orig_img.flatten()
                    list_of_imgs.append(np.asarray(flattened_img, dtype=np.float32))

                    if img_type == 1: #0 is same and 1 is different
                        list_of_same_diff.append([0,1])
                    else:
                        list_of_same_diff.append([1,0])
        
        data_imgs = np.array(list_of_imgs)
        data_img_type = np.array(list_of_same_diff)
        
        return data_imgs, data_img_type

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

def patch_next_batch(num, data, lft_lbls, rght_lbls, img_tlbl, img_key):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data[0]))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_orig = data[0, :]
    data_pred = data[1, :]
    data_orig_shuffle = [data[0, i] for i in idx]
    data_pred_shuffle = [data[1, i] for i in idx]
    lft_labels = [lft_lbls[ i] for i in idx]
    rght_labels = [rght_lbls[ i] for i in idx]
    img_tlbl = [img_tlbl[ i] for i in idx]
    img_key = [img_key[ i] for i in idx]

    return np.asarray(data_orig_shuffle), np.asarray(data_pred_shuffle), np.asarray(lft_labels), np.asarray(rght_labels), np.asarray(img_tlbl), np.asarray(img_key)

In [ ]:
def new_weights(shape, layer_name):
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.Variable(initializer(shape), name=layer_name+'_W')

def new_bias(length, layer_name):
    return tf.Variable(tf.constant(0.05, shape=[length]), name=layer_name+'_b')

def new_conv_layer(input,
                   num_input_channels,
                   filter_size,
                   num_filters,
                   name_scope,
                   layer_name='',
                   use_pooling=True):

    with tf.name_scope(name_scope):
        shape = [filter_size, filter_size, num_input_channels, num_filters]
        weights = new_weights(shape, layer_name)
        biases = new_bias(num_filters, layer_name)

        layer = tf.add(tf.nn.conv2d(input=input, filter=weights, strides=[1,1,1,1], padding='SAME'), biases, name=layer_name)

        if use_pooling:
            layer = tf.nn.max_pool(value=layer,
                                   ksize=[1, 3, 3, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME', name=layer_name+'_max')
        layer = tf.nn.relu(layer, name=layer_name+'_activation')

    return layer, weights

def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    
    return layer_flat, num_features

def new_fc_layer(input,
                num_inputs,
                num_outputs,
                name_scope,
                layer_name='',
                use_relu=True):
    
    with tf.name_scope(name_scope):
        weights = new_weights([num_inputs, num_outputs], layer_name)
        biases = new_bias(num_outputs, layer_name)

        layer = tf.add(tf.matmul(input, weights),biases,name=layer_name)
    #     layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer, layer_name+'_activation')
    
    return layer

def normalise(tensor):
    return tf.div(
   tf.subtract(
      tensor, 
      tf.reduce_min(tensor)
   ), 
   tf.subtract(
      tf.reduce_max(tensor), 
      tf.reduce_min(tensor)
   )
)

def normalized(arr):
    return (arr - np.min(arr))/(np.max(arr) - np.min(arr))

In [ ]:
# mask_graph layer configurations
m_filter_size0 = 16          # Convolution filters are 4 x 4 pixels.
m_num_filters0 = 16         # There are 16 of these filters.

m_filter_size1 = 8          # Convolution filters are 4 x 4 pixels.
m_num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
m_filter_size2 = 8          # Convolution filters are 2 x 2 pixels.
m_num_filters2 = 16         # There are 32 of these filters.

m_filter_size3 = 8          # Convolution filters are 2 x 2 pixels.
m_num_filters3 = 4         # There are 32 of these filters.

# Convolutional Layer 3.
m_filter_size4 = 4          # Convolution filters are 2 x 2 pixels.
m_num_filters4 = 32         # There are 64 of these filters.

m_filter_size5 = 2          # Convolution filters are 2 x 2 pixels.
m_num_filters5 = 16         # There are 64 of these filters.


# Fully-connected layer.
m_fc_size = 2000             # Number of neurons in fully-connected layer.

mask_graph = tf.Graph()
with mask_graph.as_default():
    m_x = tf.placeholder(tf.float32, shape=[None, img_shape[0] * img_shape[1] * img_shape[2]], name='m_x')
    m_x_image = tf.reshape(m_x, [-1, 10, 10, num_channels])
    y_true = tf.placeholder(tf.float32, shape=[None, img_shape[0] * img_shape[1]], name='m_y_true')
    y_true_cls = tf.placeholder(tf.float32, shape=[None, img_shape[0] * img_shape[1]], name='m_y_true_cls')    
    
    layer0_conv0, weights_conv0 = new_conv_layer(input=m_x_image,
                                                num_input_channels=num_channels,
                                                filter_size=m_filter_size0,
                                                num_filters=m_num_filters0,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv1',
                                                use_pooling=True)

    layer1_conv1, weights_conv1 = new_conv_layer(input=layer0_conv0,
                                                num_input_channels=m_num_filters0,
                                                filter_size=m_filter_size1,
                                                num_filters=m_num_filters1,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv2',
                                                use_pooling=True)


    layer2_conv2, weights_conv2 =  new_conv_layer(input=layer1_conv1,
                                               num_input_channels=m_num_filters1,
                                               filter_size=m_filter_size2,
                                               num_filters=m_num_filters2,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv3',
                                               use_pooling=True)

    layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
                                               num_input_channels=m_num_filters2,
                                               filter_size=m_filter_size3,
                                               num_filters=m_num_filters3,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv4',
                                               use_pooling=True)

    layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
                                               num_input_channels=m_num_filters3,
                                               filter_size=m_filter_size4,
                                               num_filters=m_num_filters4,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv5',
                                               use_pooling=True)


    layer5_conv5, weights_conv5 =  new_conv_layer(input=layer4_conv4,
                                               num_input_channels=m_num_filters4,
                                               filter_size=m_filter_size5,
                                               num_filters=m_num_filters5,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv6',
                                               use_pooling=True)

    layer_flat, num_features = flatten_layer(layer5_conv5)

    layer_fc1 = new_fc_layer(input=layer_flat,
                             num_inputs=num_features,
                             num_outputs=m_fc_size,
                             name_scope = 'mask',
                             layer_name = 'fc1',
                             use_relu=True)

    layer_fc2 = new_fc_layer(input=layer_fc1,
                             num_inputs=m_fc_size,
                             num_outputs=m_fc_size,
                             name_scope = 'mask',
                             layer_name = 'fc2',
                             use_relu=False)

    layer_fc3 = new_fc_layer(input=layer_fc2,
                             num_inputs=m_fc_size,
                             num_outputs=m_fc_size,
                             name_scope = 'mask',
                             layer_name = 'fc3',
                             use_relu=False)

    layer_fc4 = new_fc_layer(input=layer_fc3,
                             num_inputs=m_fc_size,
                             num_outputs=img_shape[0] * img_shape[1],
                             name_scope = 'mask',
                             layer_name = 'fc4',
                             use_relu=False)

    # drop_out = tf.nn.dropout(layer_fc4, 0.5, name="drop_out")

    # y_pred = tf.nn.softmax(layer_fc4, name="softmax_output")
    y_pred = layer_fc4
    
    cost = tf.reduce_mean(tf.square(y_true - y_pred))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

    # ## some more performance measures
    correct_prediction = tf.equal(y_pred, y_true)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))    

In [ ]:
sd_graph = tf.Graph()
with sd_graph.as_default():

    x = tf.placeholder(tf.float32, shape=[None, img_shape[0]*img_shape[1]*num_channels], name='x')
    x_image = tf.reshape(x, [-1, img_shape[0], img_shape[1], num_channels])
    y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
    y_true_cls = tf.argmax(y_true, axis=1)        


    #         layer1_conv1, weights_conv1 = new_conv_layer(input=x_image,
    #                                                     num_input_channels=num_channels,
    #                                                     filter_size=filter_size1,
    #                                                     num_filters=num_filters1,
    #                                                     use_pooling=True)

    #         layer2_conv2, weights_conv2 =  new_conv_layer(input=layer1_conv1,
    #                                                    num_input_channels=num_filters1,
    #                                                    filter_size=filter_size2,
    #                                                    num_filters=num_filters2,
    #                                                    use_pooling=True)

    #         layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
    #                                                    num_input_channels=num_filters2,
    #                                                    filter_size=filter_size3,
    #                                                    num_filters=num_filters3,
    #                                                    use_pooling=True)

    #         layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
    #                                                    num_input_channels=num_filters3,
    #                                                    filter_size=filter_size4,
    #                                                    num_filters=num_filters4,
    #                                                    use_pooling=True)        




    layer1_conv1, weights_conv1 = new_conv_layer(input=x_image,
                                                num_input_channels=num_channels,
                                                filter_size=filter_size1,
                                                num_filters=num_filters1,
                                                 name_scope = 'sd_graph',
                                                 layer_name = 'conv1',
                                                use_pooling=True)

    layer2_conv2, weights_conv2 = new_conv_layer(input=layer1_conv1,
                                                num_input_channels=num_filters1,
                                                filter_size=filter_size2,
                                                num_filters=num_filters2,
                                                 name_scope = 'sd_graph',
                                                 layer_name = 'conv2',
                                                use_pooling=True)


    layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
                                               num_input_channels=num_filters2,
                                               filter_size=filter_size3,
                                               num_filters=num_filters3,
                                                 name_scope = 'sd_graph',
                                                 layer_name = 'conv3',
                                               use_pooling=True)

    layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
                                               num_input_channels=num_filters3,
                                               filter_size=filter_size4,
                                               num_filters=num_filters4,
                                                 name_scope = 'sd_graph',
                                                 layer_name = 'conv4',
                                               use_pooling=True)

    layer_flat, num_features = flatten_layer(layer4_conv4)       

    layer_fc1 = new_fc_layer(input=layer_flat,
                     num_inputs=num_features,
                     num_outputs=fc_size,
                     name_scope = 'sd_graph',
                     layer_name = 'fc1',
                     use_relu=True)

    layer_fc2 = new_fc_layer(input=layer_fc1,
                             num_inputs=fc_size,
                             num_outputs=fc_size,
                             name_scope = 'sd_graph',
                             layer_name = 'fc2',
                             use_relu=False)

    layer_fc3 = new_fc_layer(input=layer_fc2,
                             num_inputs=fc_size,
                             num_outputs=fc_size,
                             name_scope = 'sd_graph',
                             layer_name = 'fc3',
                             use_relu=False)

    layer_fc4 = new_fc_layer(input=layer_fc3,
                             num_inputs=fc_size,
                             num_outputs=num_classes,
                             name_scope = 'sd_graph',
                             layer_name = 'fc4',
                             use_relu=False)

    drop_out = tf.nn.dropout(layer_fc4, 0.5)
    y_pred = tf.nn.softmax(drop_out)
    y_pred_cls = tf.argmax(y_pred, axis=1)        
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=drop_out, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))        

    #         y_pred = layer_fc4        
    #         cost = tf.reduce_mean(tf.square(y_true - y_pred))
    #         optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

    #         # ## some more performance measures
    #         correct_prediction = tf.equal(y_pred, y_true)
    #         accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
def optimize(num_epochs, save_model=True,save_name= "base_model",restore_model=False,restore_name=None):
    total_iterations = 0
    done_train_imgs = 0
    start_time = time.time()
    start_batch=0
    end_batch = train_batch_size
    plot_accuracy=[]
    plot_accuracy_epoch=[]
    plot_training_size=[]
    plot_training_size_epoch=[]
    sum_accuracy = 0.0
    n = 1
    with sd_graph.as_default():
        sd_saver = tf.train.Saver()
    
    with mask_graph.as_default():
        mask_saver = tf.train.Saver()
    
    
    with sd_graph.as_default():
        session = tf.Session(graph = sd_graph)
        session.run(tf.global_variables_initializer())

            #to save the model
        for i in range(0, num_epochs):   
            start_batch=0
            end_batch = train_batch_size

            print("Epoch:", i + 1)

            if restore_model==True:
                if restore_name==None:
                    print("No model file specified")
                    return
                else:
                    saver.restore(session,restore_name)

            sum_accuracy = 0.0
            n = 1
            while end_batch < total_imgs:

                train_data, mask_labels, left_mask, right_mask, labels = generate_batch(train_batch_size, img_shape, item_size, nitems)
#                 train, mask_labels, labels = generate_batch(train_batch_size, img_shape, item_size, nitems)
                if not len(train) and not len(labels) and not len(mask_labels):
                    print("All images have been processed.")
                    break;
                    
#                 generate left and right masks for training    
                    
                    
                    

#                 x_batch, y_true_batch = next_batch(len(train), train, labels)
                
                feed_dict_train = {x: train_data,
                           y_true: left_mask}

                session.run(optimizer, feed_dict=feed_dict_train)

                acc,co = session.run([accuracy, cost], feed_dict=feed_dict_train)
                sum_accuracy += acc
                n+=1
                msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}, Loss: {2:>.4f}"
                print(msg.format(end_batch + 1, acc, co))
                if end_batch % 100:
                    plot_accuracy.append(acc)
                    plot_training_size.append(end_batch + 1)

                start_batch += train_batch_size
                end_batch += train_batch_size

            if save_model==True:
                if save_name==None:
                    print("No model specified, model not being saved")
                    return
                else:
                    save_path = saver.save(session, save_name)
                    restore_model = True
                    print("Model saved in file: %s" % save_name)
            plot_accuracy_epoch.append(sum_accuracy/n)
            plot_training_size_epoch.append(i + 1)

        end_time = time.time()
        # Difference between start and end-times.
        time_dif = end_time - start_time

        # Print the time-usage.
        print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))  
        print(plot_accuracy)
        print(plot_training_size)
        print(plot_accuracy_epoch)
        print(plot_training_size_epoch)

In [ ]:
total_imgs = 10000
train_batch_size = 50

save_model = True
save_name = modelsd_test
restore_model=False
restore_name=modelsd_test
optimize(num_epochs=1, save_model=True,save_name=modelsd_test,restore_model=False,restore_name=modelsd_test)



In [ ]:
def see_output_grey(iNp,depth_filter_to_see=0,cmap="gray",figsize=(4,4)):
    img_x = iNp[0,:,:]
    fig = plt.figure(figsize=figsize)
    plt.imshow(img_x, interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
    plt.show()


def see_output(iNp,depth_filter_to_see=0,cmap="gray",figsize=(4,4)):
    img_x = iNp[0,:,:,:]
    fig = plt.figure(figsize=figsize)
    if cmap == "gray":
        plt.imshow(img_x, cmap=plt.get_cmap('gray'))
    else:
        plt.imshow(img_x, interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
    plt.show()